In [81]:
import pandas as pd
import numpy as np
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination

from notebooks.optimization_problems.constraints import Requirements

In [82]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/eo_telesat_post_processed.h5"

row_limit = -1

store = pd.HDFStore(file_path)

instances = store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances = instances.sort_index(0)

In [83]:
case, target = pickle.load(open('raw/eo_settings.pkl', 'rb'))

print("T_target: %.2f Gbps (min: %.2f Gbps, max: %.2f Gbps)" %
      (case['T_bitorbit_target'] / 1e9, case['T_bitorbit_min'] / 1e9, case['T_bitorbit_max'] / 1e9))
print("L_target: %.2f s" % (case['L_sorbit_target']))
print("E_target: %.2f kJ (max: %.2f kJ)" % (case['E_Jorbit_target'] / 1e3, case['E_Jorbit_max'] / 1e3))
print("P_target: %.2f s" % (case['P_sorbit_target']))

T_target: 960.00 Gbps (min: 624.00 Gbps, max: 1296.00 Gbps)
L_target: 0.00 s
E_target: 41.16 kJ (max: 82.32 kJ)
P_target: 17030.93 s


In [84]:
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.fc_Hz = target['frequency']
sys_param.GT_dBK = target['GT_dBK']
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 43.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('../dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = case['requirements']
#requirements.max_energy = -1

In [85]:
from combined_problem import ExtendedCombinedProblem
problem = ExtendedCombinedProblem(instances, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3]))

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 200, 'n_offsprings': 50, 'n_max_gen': 5000}

settings = {}

# U-NSGA-III
from pymoo.factory import get_reference_directions, get_visualization, get_termination

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)
print("# dirs: %d" % len(ref_dirs))

settings['UNSGA-III'] = {
    'label': 'unsga3',
    'algorithm': UNSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    # 'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
    #                                                   n_last=30,
    #                                                   nth_gen=5,
    #                                                   n_max_gen=algo_settings['n_max_gen'],
    #                                                   n_max_evals=None)
    'termination': get_termination("time", "01:00:00")
}

from pymoo.algorithms.rnsga2 import RNSGA2

ref_points = np.array([[case['T_bitorbit_target'], case['L_sorbit_max'], case['E_Jorbit_target'], 0.4*case['T_orbit_s']]])
ref_points = ref_points * (case['T_sim_s'] / case['T_orbit_s'])
ref_points = ref_points *  np.array([-1, 1, 1, 1])
ref_points = ref_points[:, problem.f_mask]

settings['RNSGA-II-0-001'] = {
    'label': 'rnsga2_e0_001',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.001,
        normalization='front',
        extreme_points_as_reference_points=False,
        weights=np.array([0.5, 0.5, 0])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

settings['RNSGA-II-0-01'] = {
    'label': 'rnsga2_e0_01',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.01,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

settings['RNSGA-II-0-0001'] = {
    'label': 'rnsga2_e0_0001',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.0001,
        normalization='front',
        extreme_points_as_reference_points=False,
        weights=np.array([0.5, 0.5, 0])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

setting = settings['UNSGA-III']
setting = settings['RNSGA-II-0-01']
setting = settings['RNSGA-II-0-001']
setting = settings['RNSGA-II-0-0001']

# dirs: 91


In [86]:
termination = setting['termination']
algorithm = setting['algorithm']

n_gen_inc = 500

from os import path

file_name = 'raw/eo_Telesat_%s_%d_%d.pkl' % (setting['label'], algorithm.pop_size, algorithm.n_offsprings)

if path.exists(file_name):
    problem, setting, res = pickle.load(open(file_name, 'rb'))
    algorithm = res.algorithm
    algorithm.has_terminated = False
    n_gen = res.algorithm.n_gen
else:
    n_gen = 0

In [87]:
n_gen = n_gen + n_gen_inc
print(n_gen)
termination = ('n_gen', n_gen)

500


In [ ]:
res = minimize(problem,
               algorithm,
               termination,
               #termination=termination,
               seed=1,
               #save_history=True,
               verbose=True,
               copy_algorithm=False,
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

pickle.dump((problem, setting, res), open(file_name, 'wb'))

    1 |     200 |  0.149966218 |  1.180923240 |       1 |  1.02047E+12 |        ideal
    2 |     250 |  0.00000E+00 |  1.014026654 |       1 |  4.75050E+11 |        ideal
    3 |     300 |  0.00000E+00 |  0.883849668 |       1 |  0.00000E+00 |            f
    4 |     350 |  0.00000E+00 |  0.746046908 |       1 |  0.00000E+00 |            f
    5 |     400 |  0.00000E+00 |  0.623826257 |       1 |  0.00000E+00 |            f
    6 |     450 |  0.00000E+00 |  0.518180257 |       1 |  0.00000E+00 |            f
    7 |     500 |  0.00000E+00 |  0.446060480 |       1 |  5.10102E+10 |        ideal
    8 |     550 |  0.00000E+00 |  0.380576739 |       2 |  1.000000000 |        ideal
    9 |     600 |  0.00000E+00 |  0.333755653 |       3 |  0.793995043 |        ideal
   10 |     650 |  0.00000E+00 |  0.293584213 |       5 |  0.686956522 |        ideal
   11 |     700 |  0.00000E+00 |  0.247861780 |       7 |  0.564393939 |        ideal
   12 |     750 |  0.00000E+00 |  0.205838133 |       